In [1]:
using JuMP
using MosekTools

# Initialize model
model = Model(Mosek.Optimizer)

# Index sets
days = 1:7                # i (days: Mon to Sun)
hours = 8:22              # j (hour slots from 8am to 10pm)
activities = 1:9          # k (activity types)

# Define decision variables
@variable(model, x[days, hours, activities], Bin)   # Activity scheduled
@variable(model, s[days, hours, activities], Bin)   # Activity start
@variable(model, δ[days, hours], Bin)               # Break immediately after activity
@variable(model, e[days], Bin)

# Define Constraints:
# Constraint 1: Each activity (k) can only be scheduled once per day (i) and hour (j)
for i in days, j in hours
    @constraint(model, sum(x[i,j,k] for k in activities) <= 1)
end

# Constraint 2: Each academic subject (k = 1 to 4) ≤ 2 hours per day
for i in days, k in 1:4
    @constraint(model, sum(x[i,j,k] for j in hours) <= 2)
end

# Constraint 3: consecutive hours constraint for study and for leisure

# Start detection constraints for study subjects and leisure (k ∈ {1,2,3,4,8})
for i in days, j in 9:22, k in [1, 2, 3, 4, 8]  # j starts at 9 to use j-1 = 8
    @constraint(model, x[i,j,k] - x[i,j-1,k] >= -1 + 2*s[i,j,k])
    @constraint(model, x[i,j,k] - x[i,j-1,k] <= s[i,j,k])
end

# Minimum 2-hour study session for each start (k ∈ {1,2,3,4}, j ∈ 8 to 21)
for i in days, j in 8:21, k in 1:4
    @constraint(model, sum(x[i,j+τ,k] for τ in 0:1) >= 2 * s[i,j,k])
end

# Edge case: Prevent starting 2-hour session at hour 22
for i in days, k in 1:4
    @constraint(model, s[i,22,k] == 0)
end

# Edge case: Prevent isolated 1-hour session at hour 8
for i in days, k in 1:4
    @constraint(model, x[i,8,k] <= x[i,9,k])
end

# Minimum 5-hour leisure session for each start (k = 8, i ∈ {6,7}, j ∈ 8 to 18)
for i in 6:7, j in 8:18
    @constraint(model, sum(x[i,j+τ,8] for τ in 0:4) >= 5 * s[i,j,8])
end

# Exactly one leisure session starting on the weekend
@constraint(model, sum(s[i,j,8] for i in 6:7, j in 8:18) == 1)

# Total of exactly 5 leisure hours across the week
@constraint(model, sum(x[i,j,8] for i in days, j in hours) == 5)

# Constraint 4: Required weekly hours per subject
R = Dict(1 => 12, 2 => 12, 3 => 10, 4 => 8)

for k in 1:4
    @constraint(model, sum(x[i,j,k] for i in days, j in hours) >= R[k])
end

# Constraint 5: Lunch (activity 5) occurs exactly once per day between 11am–2pm (j = 11 to 13)
for i in days
    @constraint(model, sum(x[i,j,5] for j in 11:13) == 1)
end

for i in days
    @constraint(model, sum(x[i,j,5] for j in 8:22) == 1)
end

# Constraint 6: Dinner (activity 6) occurs exactly once per day between 5pm–8pm (j = 17 to 19)
for i in days
    @constraint(model, sum(x[i,j,6] for j in 17:19) == 1)
end

for i in days
    @constraint(model, sum(x[i,j,6] for j in 8:22) == 1)
end

# Constraint 7: Exercise 

# Exactly 2 exercise sessions per week (k = 7)
@constraint(model, sum(x[i,j,7] for i in days, j in hours) == 2)

# e[i] must be 1 if any exercise on day i
for i in days
    @constraint(model, e[i] >= sum(x[i,j,7] for j in hours))
end

# No exercise on consecutive days (wrap around from Sunday to Monday)
for i in 1:6
    @constraint(model, e[i] + e[i+1] <= 1)
end
@constraint(model, e[7] + e[1] <= 1)

# Constraint 8: Preallocated Timetable Slots
@constraint(model, x[1,10,1] == 1)
@constraint(model, x[1,11,1] == 1)
@constraint(model, x[1,15,4] == 1)
@constraint(model, x[1,16,4] == 1)
@constraint(model, x[2,10,2] == 1)
@constraint(model, x[2,11,2] == 1)
@constraint(model, x[2,13,1] == 1)
@constraint(model, x[2,14,1] == 1)
@constraint(model, x[2,16,3] == 1)
@constraint(model, x[2,17,3] == 1)
@constraint(model, x[3,19,9] == 1)
@constraint(model, x[3,20,9] == 1)
@constraint(model, x[3,21,9] == 1)
@constraint(model, x[4,9,4] == 1)
@constraint(model, x[4,11,2] == 1)
@constraint(model, x[4,12,2] == 1)
@constraint(model, x[5,9,3] == 1)
@constraint(model, x[5,10,3] == 1)

# Define Objective Function
# Breaks right after activities (j = 9 to 22 because j-1 must be valid)
for i in days, j in 9:22
    @constraint(model, sum(x[i,j-1,k] for k in 1:9) - sum(x[i,j,k] for k in 1:9) >= 1 - 2*(1 - δ[i,j]))
    @constraint(model, sum(x[i,j-1,k] for k in 1:9) - sum(x[i,j,k] for k in 1:9) <= δ[i,j])
end

# Objective: Maximize total number of breaks
@objective(model, Max, sum(δ[i,j] for i in days, j in 9:22))



δ[1,9] + δ[1,10] + δ[1,11] + δ[1,12] + δ[1,13] + δ[1,14] + δ[1,15] + δ[1,16] + δ[1,17] + δ[1,18] + δ[1,19] + δ[1,20] + δ[1,21] + δ[1,22] + δ[2,9] + δ[2,10] + δ[2,11] + δ[2,12] + δ[2,13] + δ[2,14] + δ[2,15] + δ[2,16] + δ[2,17] + δ[2,18] + δ[2,19] + δ[2,20] + δ[2,21] + δ[2,22] + δ[3,9] + δ[3,10] + [[...38 terms omitted...]] + δ[5,21] + δ[5,22] + δ[6,9] + δ[6,10] + δ[6,11] + δ[6,12] + δ[6,13] + δ[6,14] + δ[6,15] + δ[6,16] + δ[6,17] + δ[6,18] + δ[6,19] + δ[6,20] + δ[6,21] + δ[6,22] + δ[7,9] + δ[7,10] + δ[7,11] + δ[7,12] + δ[7,13] + δ[7,14] + δ[7,15] + δ[7,16] + δ[7,17] + δ[7,18] + δ[7,19] + δ[7,20] + δ[7,21] + δ[7,22]

In [2]:
optimize!(model)
solution_summary(model)
value.(x) #the decision variable solution
objective_value(model) #the objective variable solution

Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : LO (linear optimization problem)
  Constraints            : 3848            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 2002            
  Matrix variables       : 0               
  Integer variables      : 2002            

Optimizer started.
Mixed integer optimizer started.
Threads used: 10
Presolve started.
Presolve terminated. Time = 0.02, probing time =  0.01
Presolved problem: 841 variables, 1061 constraints, 5648 non-zeros
Presolved problem: 0 general integer, 841 binary, 0 continuous
Clique table size: 1125
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        1        0        NA                   4.6500000000e+01     NA          0.1   
0        1        1        0        6.0000000

4987     6974     508      16       3.5000000000e+01     3.6579822515e+01     4.51        8.2   
5133     7129     520      24       3.5000000000e+01     3.6576403347e+01     4.50        8.3   
5239     7304     516      23       3.5000000000e+01     3.6570885326e+01     4.49        8.4   
5404     7480     507      13       3.5000000000e+01     3.6539745749e+01     4.40        8.5   
5482     7669     499      9        3.5000000000e+01     3.6530060099e+01     4.37        8.6   
5668     7848     477      18       3.5000000000e+01     3.6526864637e+01     4.36        8.7   
5832     8017     465      13       3.5000000000e+01     3.6494592582e+01     4.27        8.8   
5930     8231     463      19       3.5000000000e+01     3.6493271391e+01     4.27        8.9   
5930     8242     463      19       3.5000000000e+01     3.6493271391e+01     4.27        8.9   
5942     8255     463      19       3.5000000000e+01     3.6493271391e+01     4.27        8.9   
5955     8268     463      19 

35.0

In [3]:
open("schedule_output.txt", "w") do io
    for i in days, j in hours, k in activities
        if value(x[i,j,k]) > 0.9
            println(io, "Day $i, Hour $j, Activity $k scheduled")
        end
    end
end

println("✅ Full schedule saved to schedule_output.txt")


✅ Full schedule saved to schedule_output.txt


In [4]:
for i in days, j in hours, k in activities
    if value(x[i,j,k]) > 0.9
        println("Day $i, Hour $j, Activity $k scheduled")
    end
end

Day 1, Hour 8, Activity 3 scheduled
Day 1, Hour 9, Activity 3 scheduled
Day 1, Hour 10, Activity 1 scheduled
Day 1, Hour 11, Activity 1 scheduled
Day 1, Hour 13, Activity 5 scheduled
Day 1, Hour 15, Activity 4 scheduled
Day 1, Hour 16, Activity 4 scheduled
Day 1, Hour 18, Activity 6 scheduled
Day 1, Hour 20, Activity 2 scheduled
Day 1, Hour 21, Activity 2 scheduled
Day 2, Hour 8, Activity 9 scheduled
Day 2, Hour 10, Activity 2 scheduled
Day 2, Hour 11, Activity 2 scheduled
Day 2, Hour 12, Activity 5 scheduled
Day 2, Hour 13, Activity 1 scheduled
Day 2, Hour 14, Activity 1 scheduled
Day 2, Hour 16, Activity 3 scheduled
Day 2, Hour 17, Activity 3 scheduled
Day 2, Hour 19, Activity 6 scheduled
Day 2, Hour 21, Activity 4 scheduled
Day 2, Hour 22, Activity 4 scheduled
Day 3, Hour 8, Activity 2 scheduled
Day 3, Hour 9, Activity 2 scheduled
Day 3, Hour 11, Activity 5 scheduled
Day 3, Hour 13, Activity 9 scheduled
Day 3, Hour 15, Activity 7 scheduled
Day 3, Hour 17, Activity 6 scheduled
Day 3,